In [5]:
import base64
import json
from Crypto.Cipher import AES
import http.client

In [7]:
########################
airqIP = '192.168.4.1'
airqpass = 'airqsetup'
#########################

def unpad(data):
  return data[:-ord(data[-1])]


def decodeMessage(msgb64):
  # Erster Schritt: base64 dekodieren
  msg = base64.b64decode(msgb64)

  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Zweiter Schritt: AES256 dekodieren
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
  return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))

# Verbindung zum air-Q aufbauen
connection = http.client.HTTPConnection(airqIP)

# Daten anfordern
connection.request("GET", "/data")
contents = connection.getresponse()

# Daten entschlüsseln und ausgeben
msg = json.loads(contents.read())
msg['content'] = json.loads(decodeMessage(msg['content']))
print(json.dumps(msg['content']))

# Verbindung trennen
connection.close()

# Serializing json
json_object = json.dumps(msg['content'], indent=4)
 
# Writing to sample.json
with open("measurement_data.json", "w") as outfile:
    outfile.write(json_object)

{"tvoc": [26, 3.9], "pm2_5": [0, 10], "DeviceID": "de7fd19cce1c108a714ab9063e3a7518", "Status": "OK", "humidity": [51.844, 3.76], "cnt0_3": [63, 16], "measuretime": 2596, "sound": [63.1, 2.1], "temperature": [20.804, 0.54], "cnt0_5": [17, 12], "performance": 846.8409, "co": [1.485, 0.2], "humidity_abs": [9.427, 0.56], "co2": [635, 69.05], "uptime": 75309, "so2": [10.42, 145.4], "cnt2_5": [0, 10], "o3": [13.29, 1.2], "cnt10": [0, 10], "no2": [46.15, 3.9], "cnt5": [0, 10], "timestamp": 1679575110000, "pressure": [982.03, 1.0], "TypPS": 15, "cnt1": [0, 10], "door_event": 0, "pm1": [0, 10], "oxygen": [20.794, 2.44], "dewpt": [10.215, 0.9], "pm10": [0, 10], "health": 825, "bat": [0, 0], "dHdt": 0.0, "dCO2dt": 20.74}
